# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SESILIA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SESILIA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SESILIA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SESILIA\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\SESILIA\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# load data from database
database_filepath = 'DisasterProject.db'
engine = create_engine('sqlite:///../data/' + database_filepath)
df = pd.read_sql_table('DisasterResponse', engine)

X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
category_names = y.columns

In [3]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.related.unique()

array([1, 0], dtype=int64)

### 2. Write a tokenization function to process your text data

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|[?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls :
        text = text.replace(url, "urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WorNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

def tokenize(text,url_place_holder_string="urlplaceholder"):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)
    tokens = nltk.word_tokenize(text)
    stop_words = stopwords.words("english")
    tokens = [tok for tok in tokens if tok not in stop_words]
    lemmatizer = nltk.WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [7]:
def tokenize(text, url_place_holder_string="urlplaceholder"):
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words
    stop_words = stopwords.words("english")
    tokens = [tok for tok in tokens if tok not in stop_words]
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
   
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y)

#train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000021F79C2CA60>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test) # predict
class_report = classification_report(y_test, y_pred, target_names=category_names) # print classification report

print(class_report)
print('Accuracy: {}'.format(np.mean(y_test.values == y_pred))) # print accuracy score

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5011
               request       0.84      0.46      0.60      1121
                 offer       0.00      0.00      0.00        21
           aid_related       0.75      0.68      0.71      2683
          medical_help       0.62      0.07      0.13       505
      medical_products       0.70      0.08      0.15       315
     search_and_rescue       0.70      0.04      0.07       184
              security       0.33      0.01      0.02       118
              military       0.60      0.06      0.11       204
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.41      0.56       395
                  food       0.84      0.58      0.69       722
               shelter       0.83      0.33      0.48       564
              clothing       0.67      0.05      0.10        73
                 money       1.00      

C:\Users\SESILIA\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SESILIA\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SESILIA\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SESILIA\anaconda3\lib\site-packages\sklearn\metr

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estim

In [15]:
param_grid = {
    "vect__min_df": [1, 2],
    "clf__estimator__min_samples_split": [2, 6],
    "clf__estimator__max_depth": [4, 6, 8]}

cv = GridSearchCV(pipeline, param_grid, verbose=2)
cv.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=1; total time=  44.9s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=1; total time=  47.4s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=1; total time=  47.1s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=1; total time=  47.5s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=1; total time=  47.0s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=2; total time=  45.6s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=2; total time=  45.9s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_samples_split=2, vect__min_df=2; total time=  43.4s
[CV] END clf__estimator__max_depth=4, clf__estimator__min_s

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000021F79C2CA60>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_depth': [4, 6, 8],
                         'clf__estimator__min_samples_split': [2, 6],
                         'vect__min_df': [1, 2]},
             verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
y_test = cv.predict(X_test)   

In [23]:
class_report = classification_report(y_test, y_pred, target_names=category_names) # print classification report
print(class_report)

print('Accuracy: {}'.format(np.mean(y_test == y_pred))) # print accuracy score

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6554
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0
           aid_related       1.00      1.00      1.00        21
          medical_help       0.00      0.00      0.00         0
      medical_products       0.00      0.00      0.00         0
     search_and_rescue       0.00      0.00      0.00         0
              security       0.00      0.00      0.00         0
              military       0.00      0.00      0.00         0
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00         0
                  food       0.00      0.00      0.00         0
               shelter       0.00      0.00      0.00         0
              clothing       0.00      0.00      0.00         0
                 money       0.00      

C:\Users\SESILIA\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SESILIA\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
import pickle

m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.